<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/TavernAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title install and run
Model = "vKUFUzrUTxZmKaL"
#Model = "steinhaug_vKUFUzrUTxZmKaL" #remember to set to public!
Version = "Official" 
KoboldAI_Provider = "Localtunnel" #@param ["Localtunnel", "Cloudflare"]
use_google_drive = True #@param {type:"boolean"}
Provider = KoboldAI_Provider
!nvidia-smi
import subprocess, time, sys, os, threading, shutil
from google.colab import drive

if use_google_drive:
  drive.mount('/content/drive')
    
if use_google_drive:
  if not os.path.exists("/content/drive/MyDrive/TavernAI"):
    os.mkdir("/content/drive/MyDrive/TavernAI")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive"):
    os.mkdir("/content/drive/MyDrive")

def copy_characters(use_google_drive=False):
  if not use_google_drive:
    return
  
  src_folder = "/TavernAIColab/public/characters"
  dst_folder = "/content/drive/MyDrive/TavernAI/characters"

  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)

    if os.path.exists(dst_file):
      print(f"{dst_file} already exists. Skipping...")
      continue

    shutil.copy(src_file, dst_folder)
    print(f"{src_file} copied to {dst_folder}")
Revision = ""

if Model == "steinhaug_vKUFUzrUTxZmKaL":
  Model = "steinhaug/vKUFUzrUTxZmKaL"
  path = ""
  download = ""
  Version = "United"

if Model == "vKUFUzrUTxZmKaL":
  Model = "vlrfrwaprnyysf1/vKUFUzrUTxZmKaL"
  path = ""
  download = ""
  Version = "United"

if Provider == "Localtunnel":
  tunnel = "--localtunnel yes"
else:
  tunnel = ""


!wget https://koboldai.org/ckds && chmod +x ckds
!./ckds --init only
if Provider == "Localtunnel":
  p = subprocess.Popen(['/content/ckds', '--model', Model, '--localtunnel', 'yes'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
else:
  p = subprocess.Popen(['/content/ckds', '--model', Model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)


url = ''
while True:
    line = p.stdout.readline().decode().strip()
    if "KoboldAI has finished loading and is available at the following link: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link: ")[1]
        print(url)
        break
    if "KoboldAI has finished loading and is available at the following link for UI 1: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link for UI 1: ")[1]
        print(url)
        break
    if not line:
        break
    print(line)
    if "INIT" in line and "Transformers" in line:
      print("Model loading... (It will take 2 - 5 minutes)")

%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v
!git clone https://github.com/TavernAI/TavernAIColab
copy_characters(use_google_drive)
%cd TavernAIColab
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
if use_google_drive:
  %env googledrive=2
!nohup node server.js &
time.sleep(3)
print('KoboldAI LINK:')
print(url)
print('')
print('###TavernAI LINK###')
!lt --port 8000